<a href="https://colab.research.google.com/github/N47IN/Object-Detection-and-Image-Processing/blob/main/nlf_pose_estimation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# PyTorch version

import torch
import torchvision  # Must import this for the model to load without error
! git clone https://github.com/isarandi/nlf.git
! mkdir -p models
! wget -q -O models/nlf_l_multi.torchscript https://bit.ly/nlf_l_pt
model = torch.jit.load('models/nlf_l_multi.torchscript').cuda().eval()


In [ ]:

image = torchvision.io.read_image('example_image.jpg').cuda()
frame_batch = image.unsqueeze(0)

with torch.inference_mode(), torch.device('cuda'):
   pred = model.detect_smpl_batched(frame_batch)

# SMPL Parametric predictions
pred['pose'], pred['betas'], pred['trans']
pred['joints3d'], pred['vertices3d']
pred['joints2d'], pred['vertices2d']

# Nonparametric joints and vertices
pred['joints3d_nonparam'], pred['vertices3d_nonparam']
pred['joints2d_nonparam'], pred['vertices2d_nonparam']
pred['joint_uncertainties'], pred['vertex_uncertainties']

In [ ]:
vertices_list = pred['vertices3d_nonparam']

# Convert list to tensor and get desired shape
vertices_tensor = torch.stack(vertices_list)  # Shape: (1, 2, 1024, 3)
vertices_tensor = vertices_tensor.squeeze(0)[0]  # Remove batch dim and select first set: (1024, 3)

# Convert to NumPy
vertices_numpy = vertices_tensor.cpu().detach().numpy()
print(vertices_numpy.shape)  # (1024, 3)

# Create Open3D PointCloud object
point_cloud = o3d.geometry.PointCloud()
point_cloud.points = o3d.utility.Vector3dVector(vertices_numpy)

# Save the point cloud as a .ply file
o3d.io.write_point_cloud("/mnt/data/point_cloud.ply", point_cloud)

print("Point cloud saved to '/mnt/data/point_cloud.ply'")